<a href="https://colab.research.google.com/github/Airmareing/data_engineering_practice_3/blob/main/data_engineering_practice_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 1

In [22]:
import collections
import json
import os
import re
import zipfile
import pandas as pd
from bs4 import BeautifulSoup


with zipfile.ZipFile('1/zip_var_42.zip', 'r') as zip:
	zip.extractall()
items = []
for filename in os.listdir():
	if filename.endswith(".html"):
		with open(filename, "r") as f:
			html = f.read()
			sp = BeautifulSoup(html, 'html.parser')
			item = {}
			item['type'] = sp.find('div', {'class': 'chess-wrapper'}).find_all('span')[0].text.split(':')[1].strip()
			item['tournament'] = sp.find('h1', {'class': 'title'}).text.split(':')[1].strip()
			city = sp.find_all("p",attrs={"class": "address-p"})[0].get_text().replace("Город:", "")
			start = city.find("Начало")
			date = city[start:].replace("Начало:", "").strip()
			city = city[:start].strip()
			item["city"] = city
			item["start_date"] = date
			item['num_rounds'] = int(sp.find('span', {'class': 'count'}).text.split(':')[1])
			item['time_control'] = sp.find('span', {'class': 'year'}).text.split(':')[1].strip()
			item['min_rating'] = int(sp.find_all('span')[-1].text.split(':')[1])
			item['rating'] = float(sp.find_all('span')[-3].text.split(':')[1])
			item['views'] = int(sp.find_all("span", string=re.compile("Просмотры:"))[0].getText().split(':')[1].strip())
			items.append(item)

items = sorted(items, key=lambda x: x['rating'], reverse=True)
with open("1/sort.json", "w", encoding="utf-8") as f:
	f.write(json.dumps(items, ensure_ascii=False))
filtered = []
for it in items:
	if it['views'] < 50000:
		filtered.append(it)

st = []

df = pd.DataFrame(items)
pd.set_option('display.float_format', '{:.1f}'.format)

stats = df['num_rounds'].agg(['max', 'min', 'mean', 'median', 'std']).to_dict()
st.append(stats)

fr = []

words = [item['type'] for item in items]
col = collections.Counter(words)
fr.append(col)

with open("1/filter.json", "w", encoding="utf-8") as f:
	f.write(json.dumps(filtered, ensure_ascii=False))

with open("1/stats.json", "w", encoding="utf-8") as f:
	f.write(json.dumps(st, ensure_ascii=False))

with open("1/freq.json", "w", encoding="utf-8") as f:
	f.write(json.dumps(fr, ensure_ascii=False))

for filename in os.listdir():
	if filename.endswith(f'.html'):
		os.remove(filename)

Task 2

In [23]:
import collections
import json
import os
import zipfile

import pandas as pd
from bs4 import BeautifulSoup

with zipfile.ZipFile('2/zip_var_42.zip', 'r') as zip:
	zip.extractall()
items = []
for filename in os.listdir():
	if filename.endswith(".html"):
		with open(filename, "r") as f:
			text = ""
			for row in f.readlines():
				text += row

			sp = BeautifulSoup(text, 'html.parser')
			products = sp.find_all("div", attrs={'class': 'product-item'})

			for product in products:
				item = dict()
				item['id'] = product.a['data-id']
				item['link'] = product.find_all('a')[1]['href']
				item['img_url'] = product.find_all("img")[0]['src']
				item['title'] = product.find_all("span")[0].get_text().strip()
				item['price'] = int(product.price.get_text().replace("₽", "").replace(" ", "").strip())
				item['bonus'] = int(product.strong.get_text().replace("+ начислим ", "").replace(" бонусов", "").strip())

				props = product.ul.find_all("li")
				for prop in props:
					item[prop['type']] = prop.get_text().strip()

				items.append(item)

items = sorted(items, key=lambda x: x['bonus'], reverse=True)
with open("2/sort.json", "w", encoding="utf-8") as f:
	f.write(json.dumps(items, ensure_ascii=False))
filtered = []
for it in items:
	if it['price'] < 100000:
		filtered.append(it)

st = []

df = pd.DataFrame(items)
pd.set_option('display.float_format', '{:.1f}'.format)

stats = df['price'].agg(['max', 'min', 'mean', 'median', 'std']).to_dict()
st.append(stats)

fr = []

words = [item['title'] for item in items]
col = collections.Counter(words)
fr.append(col)

with open("2/filter.json", "w", encoding="utf-8") as f:
	f.write(json.dumps(filtered, ensure_ascii=False))

with open("2/stats.json", "w", encoding="utf-8") as f:
	f.write(json.dumps(st, ensure_ascii=False))

with open("2/freq.json", "w", encoding="utf-8") as f:
	f.write(json.dumps(fr, ensure_ascii=False))

for filename in os.listdir():
	if filename.endswith(f'.html'):
		os.remove(filename)

Task 3

In [26]:
import collections
import json
import os
import zipfile

import pandas as pd
from bs4 import BeautifulSoup

with zipfile.ZipFile('3/zip_var_42.zip', 'r') as zip:
	zip.extractall()
items = []
for filename in os.listdir():
	if filename.endswith(".xml"):
		with open(filename, encoding="utf-8") as f:
			text = ""
			for row in f.readlines():
				text += row

			sp = BeautifulSoup(text, 'xml')
			for it in sp.find_all("star"):
				item = {}
				for elem in it.contents:
					if elem.name == "radius":
						item[elem.name] = int(elem.get_text().strip())

					elif elem.name is not None:
						item[elem.name] = elem.get_text().strip()
				items.append(item)

items = sorted(items, key=lambda x: x['age'], reverse=True)
with open("3/sort.json", "w", encoding="utf-8") as f:
	f.write(json.dumps(items, ensure_ascii=False))

filtered = []
for constellation in items:
	if constellation['constellation'] != 'Близнецы':
		filtered.append(constellation)

st = []

df = pd.DataFrame(items)
pd.set_option('display.float_format', '{:.1f}'.format)

stats = df['radius'].agg(['max', 'min', 'mean', 'median', 'std']).to_dict()
st.append(stats)

fr = []

words = [item['name'] for item in items]
col = collections.Counter(words)
fr.append(col)

with open("3/filter.json", "w", encoding="utf-8") as f:
	f.write(json.dumps(filtered, ensure_ascii=False))

with open("3/stats.json", "w", encoding="utf-8") as f:
	f.write(json.dumps(st, ensure_ascii=False))

with open("3/freq.json", "w", encoding="utf-8") as f:
	f.write(json.dumps(fr, ensure_ascii=False))

for filename in os.listdir():
	if filename.endswith(f'.xml'):
		os.remove(filename)

Task 4

In [28]:
import collections
import json
import os
import zipfile

import pandas as pd
from bs4 import BeautifulSoup

with zipfile.ZipFile('4/zip_var_42.zip', 'r') as zip:
	zip.extractall()
items = []
for filename in os.listdir():
	if filename.endswith(".xml"):
		with open(filename, "r") as f:
			text = ""
			for row in f.readlines():
				text += row

			sp = BeautifulSoup(text, 'xml')
			for clothing in sp.find_all("clothing"):
				item = dict()
				for elem in clothing.contents:
					if elem.name is None:
						continue
					elif elem.name == "price" or elem.name == "reviews":
						item[elem.name] = int(elem.get_text().strip())
					elif elem.name == "price" or elem.name == "rating":
						item[elem.name] = float(elem.get_text().strip())
					elif elem.name == "new":
						item[elem.name] = elem.get_text().strip() == "+"
					elif elem.name == "exclusive" or elem.name == "sporty":
						item[elem.name] = elem.get_text().strip() == "yes"
					else:
						item[elem.name] = elem.get_text().strip()

				items.append(item)

items = sorted(items, key=lambda x: x['rating'], reverse=True)
with open("4/sort.json", "w", encoding="utf-8") as f:
	f.write(json.dumps(items, ensure_ascii=False))

filtered = []
for color in items:
	if color['material'] != 'Шелк':
		filtered.append(color)

st = []

df = pd.DataFrame(items)
pd.set_option('display.float_format', '{:.1f}'.format)

stats = df['price'].agg(['max', 'min', 'mean', 'median', 'std']).to_dict()
st.append(stats)

fr = []

words = [item['category'] for item in items]
col = collections.Counter(words)
fr.append(col)

with open("4/filter.json", "w", encoding="utf-8") as f:
	f.write(json.dumps(filtered, ensure_ascii=False))

with open("4/stats.json", "w", encoding="utf-8") as f:
	f.write(json.dumps(st, ensure_ascii=False))

with open("4/freq.json", "w", encoding="utf-8") as f:
	f.write(json.dumps(fr, ensure_ascii=False))

for filename in os.listdir():
	if filename.endswith(f'.xml'):
		os.remove(filename)